# Test fine tuning

Import del modello creato e aggiunta del layer softmax

In [13]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig, BertForQuestionAnswering, BertModel, BertForPreTraining

In [14]:
 
 tokenizer = BertTokenizer.from_pretrained("./transformers_models/biobert_factoid_pytorch")
 encoder = TFBertModel.from_pretrained("./transformers_models/biobert_factoid_pytorch", from_pt=True)

In [15]:
## QA Model
max_len = 62
input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
embedding = encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
)[0]

start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
start_logits = layers.Flatten()(start_logits)

end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
end_logits = layers.Flatten()(end_logits)

start_probs = layers.Activation(keras.activations.softmax)(start_logits)
end_probs = layers.Activation(keras.activations.softmax)(end_logits)

model = keras.Model(
    inputs=[input_ids, token_type_ids, attention_mask],
    outputs=[start_probs, end_probs],
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = keras.optimizers.Adam(lr=5e-5)
model.compile(optimizer=optimizer, loss=[loss, loss])

## Creazione domande

Import di alcune domande

In [16]:
question1, text1 = "List signaling molecules (ligands) that interact with the receptor EGFR?", "the epidermal growth factor receptor (EGFR) ligands, such as epidermal growth factor (EGF) and amphiregulin (AREG)"
question2, text2 = "List signaling molecules (ligands) that interact with the receptor EGFR?", "the epidermal growth factor receptor (EGFR) ligands, such as epidermal growth factor (EGF) and amphiregulin (AREG)"
dataset_raw=[[question1,text1],[question2,text2]]

In [20]:
dataset_raw[0]

['List signaling molecules (ligands) that interact with the receptor EGFR?',
 'the epidermal growth factor receptor (EGFR) ligands, such as epidermal growth factor (EGF) and amphiregulin (AREG)']

In [17]:
input_ids_list=[]
token_type_ids_list=[]
attention_mask_list=[]
start_aswer_list=[]
end_answer_list=[]

for sample in dataset_raw:
    encoding = tokenizer.encode_plus(sample[0], sample[1])
    input_ids, token_type_ids = encoding["input_ids"], encoding["token_type_ids"]
    attention_mask = [1] * len(input_ids)

    # Aggiunta manuale start token end token
    start_aswer_list.append(4)
    end_answer_list.append(5)
    
    #Altre info
    input_ids_list.append(input_ids)
    token_type_ids_list.append(token_type_ids)
    attention_mask_list.append(attention_mask)
    
input_ids_list=np.array(input_ids_list)
token_type_ids_list=np.array(token_type_ids_list)
attention_mask_list=np.array(attention_mask_list)
start_aswer_list=np.array(start_aswer_list)
end_answer_list=np.array(end_answer_list)

x_data=[input_ids_list,token_type_ids_list,attention_mask_list]
y_data=[start_aswer_list,end_answer_list]

## Training

In [18]:
model.fit(
    x_data,
    y_data,
    epochs=10,  # For demonstration, 3 epochs are recommended
    verbose=2,
)

Train on 2 samples
Epoch 1/10
2/2 - 26s - loss: 8.6288 - activation_7_loss: 4.7456 - activation_8_loss: 3.8832
Epoch 2/10
2/2 - 2s - loss: 5.0573 - activation_7_loss: 2.7610 - activation_8_loss: 2.2964
Epoch 3/10
2/2 - 2s - loss: 2.0552 - activation_7_loss: 1.1600 - activation_8_loss: 0.8953
Epoch 4/10
2/2 - 2s - loss: 0.8314 - activation_7_loss: 0.3540 - activation_8_loss: 0.4774
Epoch 5/10
2/2 - 2s - loss: 0.2729 - activation_7_loss: 0.1179 - activation_8_loss: 0.1550
Epoch 6/10
2/2 - 2s - loss: 0.0999 - activation_7_loss: 0.0521 - activation_8_loss: 0.0478
Epoch 7/10
2/2 - 2s - loss: 0.0327 - activation_7_loss: 0.0173 - activation_8_loss: 0.0154
Epoch 8/10
2/2 - 2s - loss: 0.0117 - activation_7_loss: 0.0065 - activation_8_loss: 0.0052
Epoch 9/10
2/2 - 2s - loss: 0.0157 - activation_7_loss: 0.0046 - activation_8_loss: 0.0111
Epoch 10/10
2/2 - 2s - loss: 0.0075 - activation_7_loss: 0.0028 - activation_8_loss: 0.0048
